# Information for groups working on project 7: Future Arctic sea ice in CMIP6 models

The aim of this notebook is to give you some information that might be useful for starting your project:

- Part A provides you with a quick introduction to the CMIP6 project and ensemble modelling. It also contains a list of links that provide details about the data that are available to you in the CMIP6 catalogue. You can use this to help you to decide what data you want to analyse for your project.
- Part B provides some examples of how to plot data on maps easily, by combining xarray (which you saw in Guillaume's Pangeo session) with Cartopy
- Part C gives you more specific information about this project, about sea ice, and about what you should be aiming to do. You will need to apply clustering techniques to sea ice, and perhaps other related variables, in order to characterise the Arctic sea ice evolution in different models.

Note: the installation of the libraries to plot maps using Cartopy can take a few minutes. You might want to execute the first code cell of part B now, before you read the information below, so that you don't have to wait later.

## A. Data sources and information about CMIP6

### 1. Introduction
Project 7 requires you to analyse model output from the CMIP6 project. It is possible to access these data a number of ways, including via the cloud, since Google host a part of these data via their Public Datasets programme (https://cloud.google.com/blog/products/data-analytics/new-climate-model-data-now-google-public-datasets). For your project, this is how you will access the CMIP6 data. As you have seen in class, this method means that you will be able to analyse the data directly in Colab without downloading anything to your machine.

### 2. What is ensemble modelling and why do we do it?
CMIP6 uses an ensemble modelling strategy. What does this mean?

In the context of CMIP6, an ensemble of model simulations is a collection of simulations that have been created using exactly the same configuration for everything, with the exception of the initial conditions. These small differences in the initial state can lead to much larger differences in the forecast over time, as popularised in the so-called "butterfly effect".

We use ensemble modelling to help us understand the probability of the events simulated by the model occurring. For example, imagine that we use an ensemble of 5 simulations to try to forecast next week's weather, and all 5 simulations predict that it will rain in Brest. In this case, we will be more likely to believe that it will rain next week than if our results had been that 2 of our simulations told us that it would rain, 2 told us that it would be sunny and 1 told us that it would snow.

The aim of the ensemble approach in climate modelling (as in CMIP6) is the same as in the weather forecasting case: it is an attempt to quantify how much we believe the predictions of the future climate.

### 3. What is CMIP6?
CMIP = Coupled Model Intercomparison Project (wikipedia page: https://en.wikipedia.org/wiki/Coupled_Model_Intercomparison_Project)

The CMIP programme aims to improve our knowledge of climate change. It began in 1995, and has already been through 5 "phases". Each "phase" corresponds to a coordinated effort to collect together a number of climate simulations created by research centres around the world. These simulations are then made freely available for anybody to download and analyse. Phase 6 is now underway: the data that you will be analysing are thus only recently available, and are likely to be used in research studies for about the next ten years or so.

A number of experiments are defined at each phase. These often consist of "historical" experiments, which simulate the climate over the past decade, and a number of future experiments, which simulate the future climate in response to certain prescribed greenhouse gas concentration scenarios, which should represent the response to possible future decisions made by society. (If you are interested in reading more about these, this article provides a nice introduction: https://www.carbonbrief.org/explainer-how-shared-socioeconomic-pathways-explore-future-climate-change)

### 4. What data are available for your projects?
The CMIP6 catalogue contains a lot of data: the entire archive comprises about 20PB of outputs! Not all available data is stored in the Cloud, so you will need to check to see what is available when deciding which model(s) and scenario(s) you want to analyse.

The people at Pangeo have written some useful summaries that describe:

- the data available in the Cloud: https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q/edit
- a list of the models available: https://docs.google.com/spreadsheets/d/13DHeTEH_8G08vxTMX1Fs-WbAA6SamBjDdh0fextdcGE/edit#gid=165882553
- a list of the variables available: https://docs.google.com/spreadsheets/d/1UUtoz6Ofyjlpx5LdqhKcwHFz2SGoTQV2_yekHyMfL9Y/edit#gid=1221485271

They have also produced an example notebook that shows how to load in air temperature. You can either run it via Binder on the Pangeo infrastructure: https://binder.pangeo.io/v2/gh/pangeo-data/pangeo-cmip6-examples/master or run it directly in Colab: https://colab.research.google.com/drive/19iEVxE_9QoTeg4st7MmucHJUmO93NXHp

Their notebook is an excellent introduction: it should provide you with everything that you need to get started loading the data. I recommend that you look at it!

## B. Using xarray with Cartopy

Last week we talked a lot about accessing and analysing data, but we didn't talk much about how to make figures. In the geosciences, often we want to make maps. Unfortunately, sometimes this can be more difficult than we would like! Model grids are often irregular, and it can be difficult to plot data in the polar regions, where all longitudes get closer and closer together, and finally converge to a single point.

The Pangeo notebook shows an example of loading CMIP6 data using xarray. One of the really nice features of xarray is that it integrates with the plotting library Cartopy (https://scitools.org.uk/cartopy/docs/latest/) to enable us to change map projections and easily transform our data. We'll look at an example for the Arctic below.

Let's set up the libraries that we will need, and then load in the sea surface height for the IPSL model in the Shared Socioeconomic Pathway 2 scenario.

In [ ]:
# first we have to install some libraries in Colab
!pip install zarr
!pip install gcsfs

# and set up Cartopy
!pip uninstall shapely --yes

# note: the next step produces an error in pip. You can ignore this error.
!pip install lida==0.0.10 shapely cartopy --no-binary shapely --no-binary cartopy --use-deprecated=legacy-resolver

# and manually download shape files for Cartopy
!wget https://raw.githubusercontent.com/SciTools/cartopy/main/lib/cartopy/feature/download/__main__.py -O cartopy_feature_download.py
!python cartopy_feature_download.py physical

In [ ]:
# set up our libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

# set up cartopy
import cartopy.crs as ccrs
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh

# notebook display options
xr.set_options(display_style='html')
%matplotlib inline

Now let's look at the available data in the catalogue, and select one of the simulations:

In [ ]:
# read in the catalogue information
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

# look for simulations that correspond to our chosen criteria
df_ssh = df.query("activity_id=='ScenarioMIP' & table_id == 'Omon' & variable_id == 'zos' & experiment_id == 'ssp245' & institution_id == 'IPSL'")

# print out the metadata for the available simulations in the catalogue
df_ssh

The table shows multiple available outputs for the same forcing scenario and model: these are the different members (member_id column) of the ensemble for that model and scenario. Now let's select one of the members:

In [ ]:
# setup the gcs access
gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the last one from the dataframe above)
zstore = df_ssh.zstore.values[-1]

# create a mapping interface to the store
mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated=True,decode_times=False)

The name given to the latitude and longitude variables changes depending on the model, so let's check the names of the coordinate variables for this model:

In [ ]:
ds.coords

Now we'll select only the Arctic region: we'll choose latitude > 55°N:

In [ ]:
lat = ds.nav_lat.compute()
ds=ds.where(lat > 55,drop=True)
ds

Now let's see what happens if we try to plot a map for a specific date

In [ ]:
ds.zos.sel(time=ds.time[-1]).squeeze().plot()

It's very difficult to see what's going on in the Arctic with this projection. We'll manage this by using the Polar Stereographic projection, which has the North Pole in the middle of the figure, like this:

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.NorthPolarStereo())
ax.set_extent([-180, 180, 55, 90], ccrs.PlateCarree())
ax.coastlines()

To be able to plot our data on this map, we also need to apply a transformation. We can just tell xarray to do this for us, by telling it the name of the transformation to apply:

In [ ]:
# first we set up our axes with the projection that we want
ax = plt.axes(projection=ccrs.NorthPolarStereo());
# then we set the latitude limits on our figure (here, we include all longitudes
# (-180° -> 180°) and 55°N -> 90°N)
ax.set_extent([-180, 180, 55, 90], ccrs.PlateCarree())
# now we tell xarray to plot our data. We pass a number of extra parameters:
# transform=ccrs.PlateCarree() -> tells xarray how to transform the data
# x='nav_lon' -> tells xarray the name of the longitude variable
# y='nav_lat' -> tells xarray the name of the latitude variable
ds.zos[0].plot.pcolormesh(ax=ax,transform=ccrs.PlateCarree(),x='nav_lon', y='nav_lat', add_colorbar=True);
# finally, we add the coastlines
ax.coastlines();
# and add grid lines
ax.gridlines(draw_labels=True, x_inline=False, y_inline=True,color='grey');

## C. Aims for your project

In this project you will aim to describe the evolution of the Arctic sea ice according to different models and/or radiative forcing scenarios. Arctic sea ice decrease has not been uniform over time: certain regions are losing ice much more quickly than others, and this is expected to continue into the future.

Sea ice plays a very important role in the climate system for many reasons; one of the most important is that it acts as a very effective insulating "lid" on the ocean. The ocean is warm relative to the atmospheric temperature in the Arctic, and so if the sea ice disappears, this can lead to huge changes in the quantity of heat exchanged between the atmosphere and ocean. Depending on the location at which these changes in heat transfer occur, the impact on the atmosphere and its large-scale circulation can be very different, and so it is important to understand not only how much ice is being lost, but also where the ice is being lost. Your aim in this project will be to explore these spatial differences between different models, variables, and emission scenarios.

The Arctic is divided into a number of regional seas, and sea ice change is often calculated as an average within each of these regions:
<img src='https://www.ncei.noaa.gov/monitoring-content/snow-and-ice/regional-sea-ice/images/region_definitions.png' width="500">

However, these regions are just geographical, they are not defined based on physical criteria. Your aim will be to discover how appropriate these regional definitions are for describing the future Arctic sea ice in the CMIP6 models.

Your main task in this project will be to apply clustering techniques to the time series of sea ice concentration, with the aim of identifying coherent regions of sea ice loss. You can also treat sea ice volume if you have time. You might compare the results obtained using multiple emissions scenarios to look at the consistency of the regional patterns.

### Choosing the variables to use
In the CMIP6 model outputs, you will find that a number of different variables relating to sea ice are available. The most widely available one is **"sea-ice area percentage"**: this is normally known as "sea ice concentration", and represents the percentage of each grid cell that is covered by sea ice.

Sea ice is not a 2D quantity, it also has a height, and so to better understand the processes that act on sea ice, we also use **sea ice volume**. This is not normally supplied directly by models, but can be calculated by multipling **sea ice thickness * sea ice concentration * grid-cell area for ocean variables**. You will need to retrieve these 3 variables and calculate this quantity yourselves.

Although other sea ice variables are produced by a number of models, the consistency of their availability across the different models and scenarios is not so good. Because of this, I recommend that you use only the sea ice concentration and sea ice volume variables to characterise the sea ice.

### Time averaging
**Seasonality**: it is possible that the patterns of sea ice loss will depend on the season, and so you should analyse each season separately. I recommend that you average the data as follows for each year:
* winter = December, January, February
* spring = March, April, May
* summer = June, July, August
* autumn = September, October, November

xarray can perform this averaging for you with the ```.resample``` method. An example is shown below to help you get started. It also shows you how to resample the data to have a 5-year mean for each season: this 5-year averaging is not obligatory, but it might be helpful to make this average in order to keep the data size manageable.

Once you have calculated these seasonal averages, you will then perform a separate clustering analysis for each season.

**To make the tasks simpler, you can restrict your analysis to the central Arctic region, north of 70°N**

In [ ]:
# example of accessing the sea ice concentration data and calculating
# the seasonal averages using xarray:

# first we select a model
df_tk = df.query("activity_id=='ScenarioMIP' & table_id == 'SImon' & variable_id == 'siconc' & experiment_id == 'ssp245' & institution_id == 'IPSL'")
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = df_tk.zstore.values[0]

# now we load the data into an xarray data set:
mapper = gcs.get_mapper(zstore)
ds_sic = xr.open_zarr(mapper, consolidated=True)

# we'll restrict this to the region north of 70°N again:
lat = ds_sic['nav_lat'].compute() # we use .compute() here because we need to
# actually load this data so that we can use it to index the other variables:
# xarray is naturally "lazy", and will not load the data until it is obliged to
ds_sic = ds_sic.where(lat > 70, drop=True)

# print out the metadata for this simulation: at this point we have all available
# time steps: 1 per month
ds_sic

In [ ]:
print('The simulation contains {} years of monthly data, between {} and {}'.format(ds_sic.time.dt.year.max().data-ds_sic.time.dt.year.min().data,ds_sic.time.dt.year.min().data, ds_sic.time.dt.year.max().data))

In [ ]:
# now let's make the seasonal averages, we do this using resample:
ds_sic = ds_sic.resample(time='QS-DEC').mean(dim="time")

In [ ]:
# our data has now been converted to seasonal means:
ds_sic

In [ ]:
# we can select the winter data using where
winter = ds_sic.where(ds_sic.time.dt.season == 'DJF',drop=True)
# the other seasons are called 'MAM', 'JJA', 'SON', and we can
# select them the same way

# another way to do the same thing is by taking every 4th element
# along the time dimension, like this:
# winter = ds_sic.isel(time=np.arange(0,ds_sic.time.size,4))

In [ ]:
# let's check the new data set:
ds_sic

In [ ]:
# and plot the time series at one of the points:
winter.isel(x=150,y=20).siconc.plot()

In [ ]:
# we can also make a 5-year average, to keep the overall shape
# of the sea ice decline while reducing the data volume:
winter_5y = winter.resample(time='5y').mean()
winter_5y

In [ ]:
# we now have only 19 points in the time dimension. Let's look at the time series
# at the same point as before to compare the form:
winter_5y.isel(x=150,y=20).siconc.plot()

## What next?

You should now be ready to apply a clustering analysis to the seasonal data. Start with either sea ice concentration or sea ice volume for a model and emissions scenario that you choose. How many clusters is a good choice? What do the average time series look like for each cluster? Do the clusters compare well with the geographical regions that are defined for the Arctic, or do those regions mix a number of different clusters according to your analysis? Once you have looked at one variable, you might check whether you get different results for the sea ice concentration and sea ice volume? Can you use both predictors, and how do your results change if you do?

Once you have implemented the clustering for one simulation, you have a number of choices. You can do any one, or a combination, of the following:
- analyse other ensemble members for the same model and scenario: these are simulations that are experiencing the same greenhouse gas changes and so differences from your chosen simulation will be due to "natural" climate variability as simulated by that model

- analyse different models with the same scenario: this will allow you to see differences that are due to the way that particular models simulate the sea ice. It is a way to quantify uncertainty in the future state of the Arctic that is due to the differences between models.

- analyse the same model in different emission scenarios: this will allow you to see how the ice loss spatial and temporal patterns might change as a function of greenhouse gas emissions.

For whichever option you choose, you can check whether you obtain the same spatial patterns for your clusters and average time series for each cluster. You could also see whether or not your first model can be reused (using the ```.predict``` method) to obtain the same results on this new data set, or whether it is necessary to train a new model in each case.